### Direct code to keywords generation with a code trained model

In this notebook, related to the experiment 1: we investigate whether fine-tuning of a code model on our own corpus will help in code keywords generation.

In [1]:
#%pip install protobuf==3.20.1
#%pip install -q transformers sentencepiece


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

import sys
sys.path.append('../src')

import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)

<module 'preprocessing_utils' from '/usr/net/zinovyee.hub/IRTG/Encode-the-Qode/Quantlet/code_description2project_description/../src/preprocessing_utils.py'>

In [3]:
parsed_Qs_file = '../../data/preprocessed/Quantlet/Parsed_Qs_19032023.pkl'
dataset = preprocessing_utils.QuantletDataset(parsed_Qs_file)

CLEAN_UP = True

In [22]:
model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask")
tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask",
skip_special_tokens=False)

/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [23]:
import torch

In [24]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [25]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AdamW

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

# Step 6: Fine-tuning Training Loop
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5


model.train()


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [28]:
for batch in data_loader:
    print (batch)
    break

[('rm(list = ls(all = TRUE))\n',), ('wdir = "/Users/annshchekina/Desktop/Kod/FRM_All"\n',), ('plot_labels = c(20190603, 20200101, 20200601)\n',), ('channel = "EM"\n',), ('tau = 0.50\n',), ('s = 63\n',), ('#Plot parameter defined based on the outliers\n',), ('if (channel == "Crypto") {\n',), ('  lambda_cutoff = 0.1359\n',), ('  M_macro = 5} else \n',), ('if (channel == "EM") {\n',), ('  lambda_cutoff = 10\n',), ('  M_macro = 12}\n',), ('setwd(wdir)\n',), ('if (tau == 0.05) input_path = paste0("Output/", channel) else \n',), ('  input_path = paste0("Output/", channel, "/Sensitivity/tau=", 100*tau, "/s=", s)\n',), ('output_path = input_path \n',), ('#Check if package is installed, if not: install, either way: load \n',), ('if (!require(igraph)) install.packages("igraph"); library(igraph)\n',), ('if (!require(qgraph)) install.packages("graph"); library(qgraph)\n',), ('if (!require(ggplot2)) install.packages("ggplot2"); library(ggplot2)\n',), ('if (!require(gganimate)) install.packages("gga

In [40]:
bb = '\n '.join([codeline[0].strip() for codeline in batch])

In [52]:
batch_tp = tokenized_batch.input_ids.to(device)

In [45]:
tokenized_batch = tokenizer(bb, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

In [ ]:
from transformers import AutoTokenizer, T5Model

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5Model.from_pretrained("t5-small")

input_ids = tokenizer(
    "Studies have been shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1

# preprocess: Prepend decoder_input_ids with start token which is pad token for T5Model.
# This is not needed for torch's T5ForConditionalGeneration as it does this internally using labels arg.
decoder_input_ids = model._shift_right(decoder_input_ids)

# forward pass
outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
last_hidden_states = outputs.last_hidden_state

In [ ]:



model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:

        tokenized_batch = tokenizer(bb, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
        inputs = {key: value.to(device) for key, value in batch.items()}
        
        # Forward pass
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {average_loss}")

In [8]:
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
nl_description_list = []
for batch_idx, samples in tqdm(enumerate(data_loader)):
      try:
            nl_description = pipeline([codeline[0] for codeline in samples])

In [6]:


tokenized_data = tokenizer(data, truncation=True, padding=True)

In [7]:
tokenized_data

{'input_ids': [[7089, 160, 3627, 1], [7089, 440, 3627, 1]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1]]}

In [57]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask",
    skip_special_tokens=False),
    device=0
)

/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:921: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [61]:
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

nl_description_list = []
for batch_idx, samples in tqdm(enumerate(data_loader)):
      try:
            nl_description = pipeline([codeline[0] for codeline in samples])
            if CLEAN_UP:
                  nl_description = [re.sub(r'[^a-zA-Z0-9\s]', '', summary['summary_text']) for summary in nl_description]
                  nl_description = [re.sub(r'\s+', ' ', summary).strip() for summary in nl_description]
                  nl_description = [summary for summary in nl_description if len(summary) > 0]
      except:
            nl_description = []
      nl_description_list.append({batch_idx : nl_description})

62it [01:15,  1.38it/s]Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
262it [05:07,  1.42it/s]Your max_length is set to 20, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
265it [05:09,  1.64it/s]Your max_length is set to 20, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
332it [07:08,  1.37s/it]Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
479it [09:25,  1.90it/s]Your max_length is set to 20, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
492it [09:35,  1.19it/s]Your max_length is set to 20, but you input_length is only 18. You might consider decreasing max_

In [63]:
len(nl_description_list)

8379

In [64]:
descr_Qs_file = '../../data/preprocessed/Quantlet/Descriptions_Qs_19032023.pkl'

In [65]:
with open(descr_Qs_file, 'wb') as f:
        pickle.dump(nl_description_list, f)